In [53]:
import torch

# layer 비교하기 traffic 데이터를 가져오는 식으로 비교
from layers.Embed import *

import torch.nn as nn
import torch.nn.functional as F
from layers.Transformer_EncDec import Encoder, EncoderLayer
from layers.SelfAttention_Family import FullAttention, AttentionLayer
import numpy as np

In [54]:

temporal_conv_new = TemporalConvNetNew(96, [48]*3)

In [55]:
temporal_block_new = TemporalBlockNew(96, 48, 2, 1, 1, 1, 0.2)

In [57]:
tcn_sample = TCN(96, 64, [48]*3, 2)

In [9]:
sample_input = torch.randn(2, 96, 48)

In [10]:
res_tcn = tcn_sample(sample_input)

In [11]:
res_tcn.size()

torch.Size([2, 64])

In [59]:
from data_provider.data_loader import Dataset_ETT_hour, Dataset_ETT_minute, Dataset_Custom, Dataset_Solar, Dataset_PEMS, \
    Dataset_Pred

In [60]:
from torch.utils.data import DataLoader

In [86]:
datas_set = Dataset_Custom(
    root_path = './dataset/traffic/',
    data_path = 'traffic.csv',
    flag= 'train',
    size= [96, 862, 48],
    features = 'M',
    target = 'OT',
    timeenc = 1,
    freq='h'
)

In [87]:
data_loader = DataLoader(
    datas_set,
    batch_size=16,
    shuffle=True,
    num_workers=10,
    drop_last = True
)

In [29]:
dir(datas_set)

['__add__',
 '__annotations__',
 '__class__',
 '__class_getitem__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__orig_bases__',
 '__parameters__',
 '__read_data__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_is_protocol',
 'data_path',
 'data_stamp',
 'data_x',
 'data_y',
 'features',
 'freq',
 'inverse_transform',
 'label_len',
 'pred_len',
 'root_path',
 'scale',
 'scaler',
 'seq_len',
 'set_type',
 'target',
 'timeenc']

In [73]:
len(data_loader)

755

In [88]:
model = TCN(96, 48, [862]*3, 2).float()

In [80]:
model.train()

TCN(
  (tcn): TemporalConvNetNew(
    (network): Sequential(
      (0): TemporalBlockNew(
        (conv1): Conv1d(96, 862, kernel_size=(2,), stride=(1,), padding=(1,))
        (relu1): ReLU()
        (dropout1): Dropout(p=0.2, inplace=False)
        (conv2): Conv1d(862, 862, kernel_size=(2,), stride=(1,), padding=(1,))
        (relu2): ReLU()
        (dropout2): Dropout(p=0.2, inplace=False)
        (net): Sequential(
          (0): Conv1d(96, 862, kernel_size=(2,), stride=(1,), padding=(1,))
          (1): ReLU()
          (2): Dropout(p=0.2, inplace=False)
          (3): Conv1d(862, 862, kernel_size=(2,), stride=(1,), padding=(1,))
          (4): ReLU()
          (5): Dropout(p=0.2, inplace=False)
        )
        (downsample): Conv1d(96, 862, kernel_size=(1,), stride=(1,))
        (relu): ReLU()
      )
      (1): TemporalBlockNew(
        (conv1): Conv1d(862, 862, kernel_size=(2,), stride=(1,), padding=(2,), dilation=(2,))
        (relu1): ReLU()
        (dropout1): Dropout(p=0.2,

In [81]:
sample_input = torch.randn(32, 96, 48)

In [97]:
datas_set.data_x.shape

(12280, 862)

In [82]:
model_res = model(sample_input)

In [83]:
model_res.size()

torch.Size([32, 48])

In [85]:
model_res[0]

tensor([-3.9155, -3.8848, -3.8742, -3.8775, -3.8594, -3.8667, -3.8491, -3.8309,
        -3.8454, -3.8702, -3.8828, -3.8559, -3.8646, -3.8640, -3.9193, -3.8508,
        -3.8587, -3.8605, -3.8757, -3.8337, -3.8943, -3.8446, -3.8689, -3.8793,
        -3.8998, -3.8936, -3.8846, -3.8687, -3.8616, -3.8936, -3.8455, -3.8683,
        -3.8629, -3.8768, -3.8542, -3.8927, -3.8813, -3.8745, -3.8893, -3.8872,
        -3.8710, -3.8548, -3.8626, -3.8931, -3.8674, -3.8695, -3.8800, -3.8660],
       grad_fn=<SelectBackward0>)

In [64]:
import torch.nn as nn
from torch import optim

In [65]:
model_optim = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

In [66]:
from utils.tools import EarlyStopping, adjust_learning_rate, visual

In [67]:
import time

In [68]:
time_now = time.time()

train_steps = len(data_loader)
early_stopping = EarlyStopping(patience=3, verbose=True)

In [69]:
epoch_time = time.time()
iter_count = 0

In [100]:
model_optim.zero_grad()

In [117]:
for i, (batch_x, batch_y, batch_x_mark, batch_y_mark) in enumerate(data_loader):
    iter_count += 1
    model_optim.zero_grad()
    batch_x = batch_x.float().cpu()
    batch_y = batch_y.float().cpu()
    
    # print(batch_x, batch_y)

    batch_x_mark = batch_x_mark.float().cpu()
    batch_y_mark = batch_y_mark.float().cpu()
    dec_inp = torch.zeros_like(batch_y[:, -48:, :]).float()
    dec_inp = torch.cat([batch_y[:, :862, :], dec_inp], dim=1).float().cpu()

    outputs =model(batch_x, batch_x_mark, dec_inp, batch_y_mark)[0]

    f_dim = 0
    outputs = outputs[:, -48:, f_dim:]
    batch_y = batch_y[:, -48:, f_dim:].to(0)
    
    pred = outputs.detach().cpu()
    true = batch_y.detach().cpu()

    loss = criterion(pred, true)

    print(f"{i}번째 결과")
    print(loss.item())

    
    """
    if 'PEMS' in self.args.data or 'Solar' in self.args.data:
        batch_x_mark = None
        batch_y_mark = None
    else:
        batch_x_mark = batch_x_mark.float().to(self.device)
        batch_y_mark = batch_y_mark.float().to(self.device)

    # decoder input
    dec_inp = torch.zeros_like(batch_y[:, -self.args.pred_len:, :]).float()
    dec_inp = torch.cat([batch_y[:, :self.args.label_len, :], dec_inp], dim=1).float().to(self.device)

    # encoder - decoder
    if self.args.use_amp:
        with torch.cuda.amp.autocast():
            if self.args.output_attention:
                outputs = self.model(batch_x, batch_x_mark, dec_inp, batch_y_mark)[0]
            else:
                outputs = self.model(batch_x, batch_x_mark, dec_inp, batch_y_mark)

            f_dim = -1 if self.args.features == 'MS' else 0
            outputs = outputs[:, -self.args.pred_len:, f_dim:]
            batch_y = batch_y[:, -self.args.pred_len:, f_dim:].to(self.device)
            loss = criterion(outputs, batch_y)
            train_loss.append(loss.item())
    else:
        if self.args.output_attention:
            outputs = self.model(batch_x, batch_x_mark, dec_inp, batch_y_mark)[0]
        else:
            outputs = self.model(batch_x, batch_x_mark, dec_inp, batch_y_mark)

        f_dim = -1 if self.args.features == 'MS' else 0
        outputs = outputs[:, -self.args.pred_len:, f_dim:]
        batch_y = batch_y[:, -self.args.pred_len:, f_dim:].to(self.device)
        loss = criterion(outputs, batch_y)
        train_loss.append(loss.item())

    if (i + 1) % 100 == 0:
        print("\titers: {0}, epoch: {1} | loss: {2:.7f}".format(i + 1, epoch + 1, loss.item()))
        speed = (time.time() - time_now) / iter_count
        left_time = speed * ((self.args.train_epochs - epoch) * train_steps - i)
        print('\tspeed: {:.4f}s/iter; left time: {:.4f}s'.format(speed, left_time))
        iter_count = 0
        time_now = time.time()

    if self.args.use_amp:
        scaler.scale(loss).backward()
        scaler.step(model_optim)
        scaler.update()
    else:
        loss.backward()
        model_optim.step()
    """

RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "C:\Users\Yohan\anaconda3\envs\tcitransformer\Lib\site-packages\torch\utils\data\_utils\worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
           ^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Yohan\anaconda3\envs\tcitransformer\Lib\site-packages\torch\utils\data\_utils\fetch.py", line 54, in fetch
    return self.collate_fn(data)
           ^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Yohan\anaconda3\envs\tcitransformer\Lib\site-packages\torch\utils\data\_utils\collate.py", line 264, in default_collate
    return collate(batch, collate_fn_map=default_collate_fn_map)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Yohan\anaconda3\envs\tcitransformer\Lib\site-packages\torch\utils\data\_utils\collate.py", line 142, in collate
    return [collate(samples, collate_fn_map=collate_fn_map) for samples in transposed]  # Backwards compati

In [123]:
for j in data_loader:
    print(j)

[tensor([[[ 0.1024, -0.0676,  0.9625,  ...,  0.6017,  1.5872,  1.9987],
         [ 0.0828, -0.1835, -0.5140,  ...,  0.6060,  1.4549,  1.8046],
         [ 0.1024, -0.1876, -0.5042,  ...,  0.6060,  1.0713,  1.7989],
         ...,
         [ 0.2021,  0.2519,  1.8446,  ...,  0.3309,  1.7128,  1.8731],
         [ 0.1677,  0.3780,  1.4893,  ...,  0.5157,  1.6136,  2.1014],
         [ 0.2674,  0.1339,  1.5984,  ...,  0.6490,  1.6070,  2.2042]],

        [[ 1.7394,  1.0026,  0.7027,  ...,  0.4383,  0.8001, -0.2100],
         [ 5.4432,  4.0584,  0.2408,  ...,  1.0274,  1.1308,  0.5205],
         [ 3.8127,  5.0187,  1.4526,  ...,  1.3154,  1.2432,  0.8116],
         ...,
         [-0.7161, -0.7919, -1.1107,  ..., -0.7655, -1.4354, -1.4143],
         [-0.7047, -0.7328, -1.0359,  ..., -0.7698, -1.3230, -1.4314],
         [-0.6410, -0.4846, -0.9036,  ..., -0.5591, -1.1246, -1.2602]],

        [[-0.7194, -0.7267, -1.0617,  ..., -0.7397, -1.3693, -1.4200],
         [-0.6916, -0.6271, -0.9195,  ..., -

RuntimeError: Caught RuntimeError in DataLoader worker process 1.
Original Traceback (most recent call last):
  File "C:\Users\Yohan\anaconda3\envs\tcitransformer\Lib\site-packages\torch\utils\data\_utils\worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
           ^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Yohan\anaconda3\envs\tcitransformer\Lib\site-packages\torch\utils\data\_utils\fetch.py", line 54, in fetch
    return self.collate_fn(data)
           ^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Yohan\anaconda3\envs\tcitransformer\Lib\site-packages\torch\utils\data\_utils\collate.py", line 264, in default_collate
    return collate(batch, collate_fn_map=default_collate_fn_map)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Yohan\anaconda3\envs\tcitransformer\Lib\site-packages\torch\utils\data\_utils\collate.py", line 142, in collate
    return [collate(samples, collate_fn_map=collate_fn_map) for samples in transposed]  # Backwards compati

In [108]:
batch_x.size()

torch.Size([16, 96, 862])

In [116]:
batch_x

tensor([[[-0.7276, -0.8122, -1.0874,  ..., -0.8773, -1.4619, -1.6825],
         [-0.7259, -0.7105, -1.0506,  ..., -0.8773, -1.3098, -1.6825],
         [-0.7210, -0.4216, -0.7786,  ..., -0.5032, -0.3706, -1.5912],
         ...,
         [-0.7243, -0.6820, -0.9808,  ..., -0.4301, -1.2106, -1.0490],
         [-0.7292, -0.8142, -1.0923,  ..., -0.8515, -1.3759, -1.6197],
         [-0.7292, -0.8386, -1.1119,  ..., -0.8945, -1.4553, -1.6825]],

        [[-0.5952, -0.4806, -0.7333,  ...,  0.0428, -0.5955, -0.3927],
         [-0.7210, -0.6372, -0.9183,  ..., -0.2453, -0.9328, -0.7180],
         [-0.7243, -0.7430, -1.0323,  ..., -0.4817, -1.2965, -1.0090],
         ...,
         [-0.2897, -0.4175, -0.6292,  ...,  0.8511, -0.2581,  0.6632],
         [-0.3992, -0.4907, -0.7149,  ...,  0.3911, -0.5227,  0.0696],
         [-0.5576, -0.5375, -0.7713,  ..., -0.0303, -0.7211, -0.2728]],

        [[ 0.1432, -0.0309, -0.2383,  ..., -0.6150,  0.4826,  1.0513],
         [ 0.1008,  0.0667, -0.2469,  ..., -0

In [112]:
model_batch_x = model(batch_x.cpu())

In [115]:
model_batch_x

tensor([[-3.8976, -3.8651, -3.8973, -3.8762, -3.8460, -3.8665, -3.8788, -3.8616,
         -3.8501, -3.8865, -3.8442, -3.8622, -3.8551, -3.8669, -3.8404, -3.8483,
         -3.8980, -3.8845, -3.9017, -3.8448, -3.8482, -3.8548, -3.8698, -3.8599,
         -3.9028, -3.8988, -3.8533, -3.9033, -3.9049, -3.9087, -3.8803, -3.8805,
         -3.8523, -3.8511, -3.8595, -3.8389, -3.8660, -3.9047, -3.8587, -3.8903,
         -3.8524, -3.8560, -3.8717, -3.8948, -3.8419, -3.9010, -3.8984, -3.8538],
        [-3.9004, -3.8643, -3.9040, -3.8727, -3.8473, -3.8692, -3.8825, -3.8588,
         -3.8474, -3.8891, -3.8402, -3.8575, -3.8542, -3.8712, -3.8400, -3.8451,
         -3.8964, -3.8830, -3.9052, -3.8451, -3.8529, -3.8518, -3.8694, -3.8594,
         -3.9028, -3.8975, -3.8520, -3.9052, -3.9034, -3.9144, -3.8811, -3.8775,
         -3.8495, -3.8461, -3.8628, -3.8404, -3.8646, -3.9132, -3.8604, -3.8901,
         -3.8527, -3.8479, -3.8718, -3.8919, -3.8414, -3.9006, -3.9026, -3.8531],
        [-3.9004, -3.8593,